Домашнее задание по теме «Многослойная нейронная сеть»<br>
Цель задания: <br>
научиться на простейшем уровне подбирать архитектуру полносвязной нейронной сети для получения качества решения задачи не ниже заданного.
<br><br>
Задание<br>
Постройте модель на основе полносвязных слоёв для классификации Fashion MNIST из библиотеки torchvision (datasets).<br>
Получите качество на тестовой выборке не ниже 88%
<br><br>
Инструкция по выполнению задания
<br><br>
Скачайте тренировочную и тестовою часть датасета Fashion MNIST<br>
Постройте модель, выбрав стартовую архитектуру<br>
Обучите модель и сверьте качество на тестовой части с заданным порогом<br>
Изменяйте архитектуру модели пока качество на тестовой части не будет выше порога. <br>Вариации архитектуры можно реализовать через изменение количества слоёв, количества нейронов в слоях и использование регуляризации.<br> Можно использовать различные оптимизаторы.<br>
<br>
Критерии оценки<br>
Задание считается выполненным, если:<br><br>

модель обучается<br>
модель показала качество на тесте более 88%<br>

первая строка  
вторая строка  
два пробела и ентер  


In [ ]:
import pandas as pd
import numpy as np

import time


import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, random_split

import torchvision as tv

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
BATCH_SIZE=256

In [ ]:
train_dataset = tv.datasets.FashionMNIST(root = '.', train = True, download=True, transform=tv.transforms.ToTensor())
test_dataset = tv.datasets.FashionMNIST(root = '.', train = False, download=True, transform=tv.transforms.ToTensor())

100%|██████████| 26421880/26421880 [00:01<00:00, 18226882.68it/s]


Extracting ./FashionMNIST/raw/train-images-idx3-ubyte.gz to ./FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 310604.61it/s]


Extracting ./FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 5531922.54it/s]


Extracting ./FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 4267248.42it/s]

Extracting ./FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./FashionMNIST/raw



In [ ]:
train = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE)
test = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [ ]:
train_dataset[0][0].shape

torch.Size([1, 28, 28])

In [ ]:
28*28

784

In [ ]:
model = torch.nn.Sequential(
    torch.nn.Flatten(), # вытягиваем в вектор
    torch.nn.Linear(784, 256), # полносвязный слой с входом 28х28=784 и выходом 256
    torch.nn.ReLU(), # функция активации
    torch.nn.Linear(256, 10) # полносвязный слой с входом 256 и выходом 10 - каждый нейрон отвечает за свою цифру/класс
)

In [ ]:
model

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=256, bias=True)
  (2): ReLU()
  (3): Linear(in_features=256, out_features=10, bias=True)
)

In [ ]:
loss = torch.nn.CrossEntropyLoss() # функция потерь для классификации
trainer = torch.optim.SGD(model.parameters(), lr=.01) # оптимайзер функции потерь (передали туда параметры и шаг обучения) - схокастический градиентный спуск
num_epochs = 10 # кол-во эпох

In [ ]:
 # обучаем модель
def train_model():
    for ep in range(num_epochs):
        train_iters, train_passed  = 0, 0 # инициализируем параметры для процесса обучения
        train_loss, train_acc = 0., 0.
        start=time.time() # фиксируем время начала обучения

        model.train() # переводим модель в режим обучения (чтобы использовать инструментарий типа dropout и bacth normalization)
        for X, y in train:  # итерируемся по данным
            trainer.zero_grad() # обнуляем старые значения градиентов
            y_pred = model(X) # прогоняем данные через модель, получаем предсказания
            l = loss(y_pred, y) # вызываем функцию потерь для получения и оценки ошибки (сравниваем с истинными размеченными значениями y - позиция 1 в строке от 0 до 9)
            l.backward() # считаем градиенты
            trainer.step() # делаем шаг градиентного спуска
            train_loss += l.item() # суммируем среднее значение функции потерь на каждом элементе этого батча в этой иттерации
            train_acc += (y_pred.argmax(dim=1) == y).sum().item() # находим максимальную (по предсказанной вероятности) позицию в предсказании и сравниваем с позицией верного ответа
            # суммируем матрицу и получаем кол-во правильно предсказанных символов - это и есть почти аккураси в этом случае
            # т.к. True=1 а False=0
            train_iters += 1 # сохраняем кол-во иттераций
            train_passed += len(X) # сохраняем, сколько данных прогнали через модель

 # делаем такой же цикл для тестовых данных
        test_iters, test_passed  = 0, 0
        test_loss, test_acc = 0., 0.
        model.eval() # переводим в режим исполнения
        for X, y in test:
           # не обнуляем старые значения градиентов
            y_pred = model(X)
            l = loss(y_pred, y)
             # не считаем градиенты и не делаем шаг градиентного спуска
            test_loss += l.item()
            test_acc += (y_pred.argmax(dim=1) == y).sum().item()
            test_iters += 1
            test_passed += len(X)
 # выводим промежуточные результаты за эпоху
        print("ep: {}, taked: {:.3f}, train_loss: {}, train_acc: {}, test_loss: {}, test_acc: {}".format(
            ep, time.time() - start, train_loss / train_iters, train_acc / train_passed,
            test_loss / test_iters, test_acc / test_passed)
        )

In [ ]:
%%time
train_model()

ep: 0, taked: 9.401, train_loss: 1.7227666418603127, train_acc: 0.5743166666666667, test_loss: 1.2695170968770981, test_acc: 0.6527
ep: 1, taked: 10.436, train_loss: 1.0629078309586708, train_acc: 0.6712833333333333, test_loss: 0.9384632349014282, test_acc: 0.6734
ep: 2, taked: 10.513, train_loss: 0.8600352089455787, train_acc: 0.70425, test_loss: 0.8161783218383789, test_acc: 0.7082
ep: 3, taked: 10.200, train_loss: 0.7689622932292045, train_acc: 0.7349666666666667, test_loss: 0.747823429107666, test_acc: 0.7349
ep: 4, taked: 9.615, train_loss: 0.7119598233953436, train_acc: 0.7568166666666667, test_loss: 0.7005664236843586, test_acc: 0.7541
ep: 5, taked: 10.500, train_loss: 0.6703068266523645, train_acc: 0.77245, test_loss: 0.6648170277476311, test_acc: 0.769
ep: 6, taked: 10.459, train_loss: 0.6378206899825563, train_acc: 0.78585, test_loss: 0.63672701343894, test_acc: 0.7796
ep: 7, taked: 9.399, train_loss: 0.611764579757731, train_acc: 0.79625, test_loss: 0.6141807787120342, test_

<font color=green size=3>
Получили вполне приличный результат
<br>
train_acc: 0.80975, test_acc: 0.7988
<br>
на самой простейшей сети
</font>

In [ ]:
%%time
trainer = torch.optim.Adam(model.parameters(), lr=.01)
train_model()

ep: 0, taked: 10.206, train_loss: 0.5789127268689744, train_acc: 0.8133, test_loss: 0.4507943294942379, test_acc: 0.8306
ep: 1, taked: 11.018, train_loss: 0.38555354158929056, train_acc: 0.8602666666666666, test_loss: 0.40124305970966817, test_acc: 0.8566
ep: 2, taked: 10.869, train_loss: 0.3495318640419777, train_acc: 0.8714166666666666, test_loss: 0.3855059117078781, test_acc: 0.8597
ep: 3, taked: 11.459, train_loss: 0.33310477492657115, train_acc: 0.8771, test_loss: 0.3981715518981218, test_acc: 0.8548
ep: 4, taked: 9.960, train_loss: 0.3162609485869712, train_acc: 0.88365, test_loss: 0.3847768548876047, test_acc: 0.8608
ep: 5, taked: 11.319, train_loss: 0.30223311540928294, train_acc: 0.8883166666666666, test_loss: 0.382225776091218, test_acc: 0.8701
ep: 6, taked: 11.131, train_loss: 0.2930746440557723, train_acc: 0.8917166666666667, test_loss: 0.4010060906410217, test_acc: 0.8624
ep: 7, taked: 11.093, train_loss: 0.2841165393590927, train_acc: 0.8946166666666666, test_loss: 0.4126

<font color=green size=3>
Применение более продвинутого оптимизатора
<br>
на самой простейшей сети
<br>
было: train_acc: 0.80975, test_acc: 0.7988
<br>
стало: train_acc: 0.89383, test_acc: 0.8696
<br>
значительно улучшило результат
<br>
скорость выполнения замедлилась примерно на 1 сек на слой
<br>
<br>
Добавим еще полносвязных слоев
</font>

In [ ]:
model = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(784, 512),
    torch.nn.ReLU(),
    torch.nn.Linear(512, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 128),
    torch.nn.ReLU(),
    torch.nn.Linear(128, 10)
)

In [ ]:
model

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=512, bias=True)
  (2): ReLU()
  (3): Linear(in_features=512, out_features=256, bias=True)
  (4): ReLU()
  (5): Linear(in_features=256, out_features=128, bias=True)
  (6): ReLU()
  (7): Linear(in_features=128, out_features=10, bias=True)
)

In [ ]:
%%time
trainer = torch.optim.Adam(model.parameters(), lr=.01)
train_model()

ep: 0, taked: 13.652, train_loss: 0.5947571552814321, train_acc: 0.7814, test_loss: 0.4532568156719208, test_acc: 0.8332
ep: 1, taked: 13.733, train_loss: 0.39248946030089193, train_acc: 0.8547333333333333, test_loss: 0.40977087095379827, test_acc: 0.8506
ep: 2, taked: 14.467, train_loss: 0.35587547186841356, train_acc: 0.8693333333333333, test_loss: 0.38904671296477317, test_acc: 0.8616
ep: 3, taked: 18.636, train_loss: 0.33322388897550864, train_acc: 0.8766166666666667, test_loss: 0.37915405035018923, test_acc: 0.8593
ep: 4, taked: 17.162, train_loss: 0.33077862497339855, train_acc: 0.8789166666666667, test_loss: 0.39231794103980067, test_acc: 0.8588
ep: 5, taked: 15.928, train_loss: 0.3133512649764406, train_acc: 0.8849166666666667, test_loss: 0.39937999844551086, test_acc: 0.8612
ep: 6, taked: 15.382, train_loss: 0.3069969519021663, train_acc: 0.8861, test_loss: 0.39864862263202666, test_acc: 0.8611
ep: 7, taked: 15.658, train_loss: 0.30354485029869893, train_acc: 0.888416666666666

<font color=green size=3>
Дополнительные полносвязные слои
<br>
улучшили качество сети незначительно
<br>
было: train_acc: 0.80975, test_acc: 0.7988
<br>
было: train_acc: 0.89383, test_acc: 0.8696
<br>
стало: train_acc: 0.89392, test_acc: 0.8711
<br>
скорость выполнения замедлилась примерно на одну треть
<br>
<br>
Для улучшения модели добавим слои нормализации
</font>

In [ ]:
model = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(784, 512),
    torch.nn.BatchNorm1d(512),
    torch.nn.ReLU(),
    torch.nn.Linear(512, 256),
    torch.nn.BatchNorm1d(256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 128),
    torch.nn.BatchNorm1d(128),
    torch.nn.ReLU(),
    torch.nn.Linear(128, 10)
)

In [ ]:
%%time
trainer = torch.optim.Adam(model.parameters(), lr=.01)
train_model()

ep: 0, taked: 14.502, train_loss: 0.4699707555009964, train_acc: 0.8267333333333333, test_loss: 0.4248347692191601, test_acc: 0.8344
ep: 1, taked: 14.218, train_loss: 0.3467641055583954, train_acc: 0.8720666666666667, test_loss: 0.39942176584154365, test_acc: 0.8531
ep: 2, taked: 14.295, train_loss: 0.3042967079801762, train_acc: 0.8878666666666667, test_loss: 0.40805117608979347, test_acc: 0.85
ep: 3, taked: 14.287, train_loss: 0.27624466831379746, train_acc: 0.8978833333333334, test_loss: 0.3563946672715247, test_acc: 0.8685
ep: 4, taked: 14.110, train_loss: 0.2520161214026999, train_acc: 0.90585, test_loss: 0.36648095566779376, test_acc: 0.8697
ep: 5, taked: 14.252, train_loss: 0.23250088006892103, train_acc: 0.9132, test_loss: 0.3650816443376243, test_acc: 0.8717
ep: 6, taked: 14.066, train_loss: 0.21577474639770833, train_acc: 0.9182333333333333, test_loss: 0.37629660964012146, test_acc: 0.874
ep: 7, taked: 14.227, train_loss: 0.2018246056551629, train_acc: 0.9247, test_loss: 0.39

<font color=green size=3>
Добавление слоев нормализации
<br>
улучшило качество сети
<br>
было: train_acc: 0.80975, test_acc: 0.7988
<br>
было: train_acc: 0.89383, test_acc: 0.8696
<br>
было: train_acc: 0.89392, test_acc: 0.8711
<br>
стало: train_acc: 0.93212, test_acc: 0.8742
<br>
скорость выполнения даже незначительно улучшилась
<br>
но стало настолько хорошо, что на тренировочных данных похоже что наступает переобучение
<br>
Посмотрим, сможем ли мы достичь требуемого результата 0.88 просто увеличив число эпох в 2 раза
</font>

In [ ]:
num_epochs = 20

In [ ]:
%%time
trainer = torch.optim.Adam(model.parameters(), lr=.01)
train_model()

ep: 0, taked: 14.098, train_loss: 0.18096288876964692, train_acc: 0.9312, test_loss: 0.3943815511884168, test_acc: 0.8724
ep: 1, taked: 13.953, train_loss: 0.1597659133533214, train_acc: 0.9385833333333333, test_loss: 0.3844720114575466, test_acc: 0.8831
ep: 2, taked: 14.020, train_loss: 0.15284970107864826, train_acc: 0.9412, test_loss: 0.4779805944068357, test_acc: 0.8593
ep: 3, taked: 14.113, train_loss: 0.1415883183796355, train_acc: 0.9463166666666667, test_loss: 0.46806846123654394, test_acc: 0.8685
ep: 4, taked: 14.410, train_loss: 0.13273154015236713, train_acc: 0.9492166666666667, test_loss: 0.49027262661547866, test_acc: 0.8679
ep: 5, taked: 14.260, train_loss: 0.12112936215831878, train_acc: 0.95355, test_loss: 0.5097814704407938, test_acc: 0.8672
ep: 6, taked: 14.286, train_loss: 0.11202544975788035, train_acc: 0.9572166666666667, test_loss: 0.5215007632854395, test_acc: 0.8737
ep: 7, taked: 14.459, train_loss: 0.1079553929573678, train_acc: 0.9593833333333334, test_loss: 0

<font color=green size=3>
Увеличение кол-ва эпох в 2 раза
<br>
позволило нам достичь требуемой точности предсказания сети в 88%
<br>
было: train_acc: 0.80975, test_acc: 0.7988
<br>
было: train_acc: 0.89383, test_acc: 0.8696
<br>
было: train_acc: 0.89392, test_acc: 0.8711
<br>
было: train_acc: 0.93212, test_acc: 0.8742
<br>
стало: train_acc: 0.97815, test_acc: 0.8817
<br>
времени это заняло в 2 раза больше
<br>
переобучение на тренировочных данных помешало нам не сильно
<br>
Посмотрим, могли ли мы улучшить результат просто применив слой Dropout
</font>

In [ ]:
model = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(784, 512),
    torch.nn.Dropout(0.5),
    torch.nn.BatchNorm1d(512),
    torch.nn.ReLU(),
    torch.nn.Linear(512, 256),
    torch.nn.BatchNorm1d(256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 128),
    torch.nn.BatchNorm1d(128),
    torch.nn.ReLU(),
    torch.nn.Linear(128, 10)
)

In [ ]:
model

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=512, bias=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (4): ReLU()
  (5): Linear(in_features=512, out_features=256, bias=True)
  (6): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (7): ReLU()
  (8): Linear(in_features=256, out_features=128, bias=True)
  (9): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (10): ReLU()
  (11): Linear(in_features=128, out_features=10, bias=True)
)

In [ ]:
num_epochs = 10

In [ ]:
%%time
trainer = torch.optim.Adam(model.parameters(), lr=.01)
train_model()

ep: 0, taked: 14.260, train_loss: 0.5488258606575905, train_acc: 0.7957833333333333, test_loss: 0.43133625350892546, test_acc: 0.8429
ep: 1, taked: 14.150, train_loss: 0.424601237951441, train_acc: 0.8437333333333333, test_loss: 0.3989048205316067, test_acc: 0.8566
ep: 2, taked: 14.246, train_loss: 0.39307717478021664, train_acc: 0.8538833333333333, test_loss: 0.3974382795393467, test_acc: 0.8537
ep: 3, taked: 14.326, train_loss: 0.3686343227295165, train_acc: 0.8635333333333334, test_loss: 0.41176957711577417, test_acc: 0.8468
ep: 4, taked: 14.208, train_loss: 0.3522034965931101, train_acc: 0.8694833333333334, test_loss: 0.393422681465745, test_acc: 0.8543
ep: 5, taked: 14.275, train_loss: 0.33778318096982674, train_acc: 0.8744166666666666, test_loss: 0.4011519458144903, test_acc: 0.8525
ep: 6, taked: 14.191, train_loss: 0.32470212249045677, train_acc: 0.8794333333333333, test_loss: 0.39112487714737654, test_acc: 0.8574
ep: 7, taked: 15.095, train_loss: 0.31779362565659464, train_acc:

<font color=green size=3>
Добавление слоя Dropout
<br>
убрало проблему переобучения, но требуемого параметра точности не достигли
<br>
было: train_acc: 0.80975, test_acc: 0.7988
<br>
было: train_acc: 0.89383, test_acc: 0.8696
<br>
было: train_acc: 0.89392, test_acc: 0.8711
<br>
было: train_acc: 0.93212, test_acc: 0.8742
<br>
стало: train_acc: 0.888, test_acc: 0.8507
<br>
скорость выполнения не изменилась
<br>
Посмотрим, изменится ли что нибудь, если мы добавим слой Dropout ближе к голове сети, где связей меньше, а информация "плотнее"
</font>

In [ ]:
model = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(784, 512),
    torch.nn.BatchNorm1d(512),
    torch.nn.ReLU(),
    torch.nn.Linear(512, 256),
    torch.nn.BatchNorm1d(256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 128),
    torch.nn.Dropout(0.5),
    torch.nn.BatchNorm1d(128),
    torch.nn.ReLU(),
    torch.nn.Linear(128, 10)
)

In [ ]:
model

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=512, bias=True)
  (2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (3): ReLU()
  (4): Linear(in_features=512, out_features=256, bias=True)
  (5): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): ReLU()
  (7): Linear(in_features=256, out_features=128, bias=True)
  (8): Dropout(p=0.5, inplace=False)
  (9): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (10): ReLU()
  (11): Linear(in_features=128, out_features=10, bias=True)
)

In [ ]:
%%time
trainer = torch.optim.Adam(model.parameters(), lr=.01)
train_model()

ep: 0, taked: 14.279, train_loss: 0.5146541495272454, train_acc: 0.81735, test_loss: 0.4185206709429622, test_acc: 0.8446
ep: 1, taked: 15.407, train_loss: 0.3761032136830878, train_acc: 0.8660666666666667, test_loss: 0.41394062526524067, test_acc: 0.8479
ep: 2, taked: 14.232, train_loss: 0.3276637629625645, train_acc: 0.88175, test_loss: 0.40454412456601857, test_acc: 0.8553
ep: 3, taked: 14.302, train_loss: 0.3012321828527654, train_acc: 0.8918833333333334, test_loss: 0.3805054547265172, test_acc: 0.862
ep: 4, taked: 14.518, train_loss: 0.2756936955959239, train_acc: 0.9011166666666667, test_loss: 0.36149481013417245, test_acc: 0.8715
ep: 5, taked: 14.210, train_loss: 0.2581388068960068, train_acc: 0.90515, test_loss: 0.35508041260764, test_acc: 0.8738
ep: 6, taked: 14.181, train_loss: 0.24271697135681802, train_acc: 0.911, test_loss: 0.34506201101467016, test_acc: 0.881
ep: 7, taked: 14.194, train_loss: 0.22505057783837015, train_acc: 0.9177666666666666, test_loss: 0.374209173955023

<font color=green size=3>
Добавление слоя Dropout в конец сети
<br>
повлияло на проблему переобучения не так сильно, но зато мы на 10 эпохах достигли требуемого параметра точности в 88%
<br>
было: train_acc: 0.80975, test_acc: 0.7988
<br>
было: train_acc: 0.89383, test_acc: 0.8696
<br>
было: train_acc: 0.89392, test_acc: 0.8711
<br>
было: train_acc: 0.93212, test_acc: 0.8742
<br>
стало: train_acc: 0.888, test_acc: 0.8507
<br>
стало: train_acc: 0.92858, test_acc: 0.882
<br>
скорость выполнения не изменилась
<br>
Можно сделать вывод, что изменение слоев ближе к голове сети сильнее влияет на результат ее работы.
<br>
А в целом, главное выдержать правильный баланс.
</font>

Хотелось бы порекомендовать использовать функцию активации выходного слоя. На практике обучение классификатора возможно и без нее, но это хорошая практика, которая иногда позволяет добиться лучших результатов и избежать некоторых проблем, например т.н. “градиентный взрыв”.
<br><br>
Для бинарной классификации обычно рекомендуют использовать Sigmoid, для многоклассовой - Softmax (хотя можно пробовать и другие варианты):
<br><br>
	nn.Sequential(<br><br>
	nn.Flatten(),<br>
	nn.Linear(784, 256),<br>
	nn.ReLU(),<br>
	nn.Linear(256, 10),<br>
	nn.Softmax()<br>
)<br><br>
Линейные слои могут возвращать любые значения, как положительные, так и отрицательные, а также очень большие, в то время как softmax и sigmoid нормируют их в диапазон от 0 до 1.

<br>

<font color=green size=3>
Это шаблон примечания
<br>
<br>
</font>